---
  title: "RestFull 06: Concepts avancés"
  description: "Providers, Schemas and Filters with Jakarta RESTful Web Services."
  categories: 
    - Java
    - Lecture
    - RestFull
  provide_notes: true
  provide_slides: false
  jupyter: java-lts
  echo: true
  output: true
---


{{< embed ./quarto-utils/_version.qmd >}}

In [1]:
//| output: false
//| echo: false

%printWithName false
    
// UPDATE SAMPLE SOURCE CODE

String script="""
GITHUB_REPO=ebpro/sample-jaxrs
GITHUB_URL=https://github.com/${GITHUB_REPO}
BRANCH=develop
SRC_DIR=/home/jovyan/work/src/github/${GITHUB_REPO}
gitpuller ${GITHUB_URL} ${BRANCH} ${SRC_DIR}
cd ${SRC_DIR}
mvn --quiet clean package
""";
IJava.getKernelInstance().getMagics().applyCellMagic("shell",List.of(""),script);  

$ git fetch


$ git reset --mixed


$ git -c user.email=nbgitpuller@nbgitpuller.link -c user.name=nbgitpuller merge -Xours origin/develop


Already up to date.


0

In [2]:
//| output: false
//| echo: false
%jars "/home/jovyan/work/src/github/ebpro/sample-jaxrs/target/sample-jaxrs-*-withdependencies.jar"; 
import org.glassfish.grizzly.http.server.HttpServer;
import fr.univtln.bruno.samples.jaxrs.server.BiblioServer;
HttpServer httpServer = BiblioServer.startServer();
httpServer.toString();

org.glassfish.grizzly.http.server.HttpServer@2953ea1b


# Les liens sémantiques

Les liens sémantiques entre les resources peuvent être réprésentés à l'aide d'URLs de façon standard ((RFC8288)[https://tools.ietf.org/html/rfc8288]). Une liste complète est données sur le site de l'IANA : https://www.iana.org/assignments/link-relations/link-relations.xhtml. Les plus classiquement utilisé avec REST sont self, first, previous, next et last. Pour simplifier l'usage de l'API ces liens peuvent être fournis dans l'entête d'une réponse http et/ou dans le corps de la réponse (par exemple lors de la pagination).

```http
Link: <http://MyServer:8080/MyApp/Persons/1>; rel="first-person", <http://MyServer:8080/MyApp/Persons/10>; rel="next-person", <http://MyServer:8080/MyApp/Persons/8>; rel="previous-person", <http://MyServer:8080/MyApp/Persons/90>; rel="last-person"
```

## Etendre JAX-RS avec les Providers

D'une façon générale une classe annotée par `@Provider` ajoute des fonctions à JAXRS (traitement des exceptions, conversion des données, ...).

#### Les mappers : application aux exceptions
Le traitement des exceptions peut être simplifié en utilisant des mappers (cf. package exceptions et mapper) qui s'appliquent automatiquement lors qu'une exception est émise. Dans ce cas un objet Response est construit manuellement pour contrôler le détail du corps et de l'entête. 

In [3]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/exceptions/NotFoundException.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcClassByName",List.of("NotFoundException"),script);
return null;


```Java
@XmlRootElement
public class NotFoundException extends BusinessException {
    public NotFoundException() {
        super(Response.Status.NOT_FOUND);
    }
}
```

In [4]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/mappers/BusinessExceptionMapper.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcClassByName",List.of("BusinessExceptionMapper"),script);
return null;


```Java
@SuppressWarnings("unused")
@Provider
@FieldDefaults(level = AccessLevel.PRIVATE)
@Log
public class BusinessExceptionMapper implements ExceptionMapper<BusinessException> {
    public Response toResponse(BusinessException ex) {
        return Response.status(ex.getStatus())
                .entity(ex)
                .build();
    }
}
```

If a resource doesn't exist an exception is raised, and the 404 http status code is returned

In [5]:
%%shell
curl -s -i -H "Accept: application/json"  \
  http://localhost:9998/mylibrary/authors/1000

HTTP/1.1 404 Not Found


Content-Type: application/json


Content-Length: 89


{"status":"NOT_FOUND","message":"Author not found","localizedMessage":"Author not found"}


## Les filtres : application au statut et à la pagination

In [6]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/status/StatusFilter.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcClassByName",List.of("StatusFilter"),script);
return null;



```Java
@Provider
public class StatusFilter implements ContainerResponseFilter {

    @Override
    public void filter(ContainerRequestContext containerRequestContext, ContainerResponseContext containerResponseContext) {
        if (containerResponseContext.getStatus() == Response.Status.OK.getStatusCode()) {
            for (Annotation annotation : containerResponseContext.getEntityAnnotations()) {
                if (annotation instanceof Status) {
                    containerResponseContext.setStatus(((Status) annotation).value());
                    break;
                }
            }
        }
    }

}
```

In [7]:
%%shell
cat /home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/status/Status.java

package fr.univtln.bruno.samples.jaxrs.status;


import jakarta.ws.rs.NameBinding;


import java.lang.annotation.Retention;


import java.lang.annotation.RetentionPolicy;


@NameBinding


...

    int NO_CONTENT = 204;


    int RESET_CONTENT = 205;


    int PARTIAL_CONTENT = 206;


    int value();


}


Un exemple de filtre appliqué à la requête sera donné dans la partie sur la sécurité.

## Schéma d'une API REST

  * [WSDL](https://www.w3.org/TR/wsdl/) un autre langage de service Web permet de décrire un tel [schéma](https://www.ibm.com/developerworks/webservices/library/ws-restwsdl/).
  * Le langage [WADL](https://www.w3.org/Submission/wadl/) est spécifique à REST. 
  * Par défaut Jersey génére une description WADL `/application.wadl`
  * Cependant, pour cela l'application nécessite une implantation du standard pour traiter les documents XML appelé [JAXB](https://eclipse-ee4j.github.io/jaxb-ri/) qui n'est plus fournie en standard dans le JDK depuis la version 8 (nous utilisons ici la version 15).

Si JAXB est dans le classpath (cf. pom.xml) la génération d'une description WADL est automatique.

In [8]:
%%shell
curl -s -D - --get http://localhost:9998/mylibrary/application.wadl

HTTP/1.1 200 OK


Last-modified: Thu, 03 Oct 2024 13:07:57 UTC


Content-Type: application/vnd.sun.wadl+xml


Transfer-Encoding: chunked


<?xml version="1.0" encoding="UTF-8" standalone="yes"?>


...

                </method>


            </resource>


        </resource>


    </resources>


</application>


https://jakarta.ee/specifications/restful-ws/3.0/jakarta-restful-ws-spec-3.0.html#context


In [9]:
//| output: false
//| echo: false
httpServer.stop();